In [1]:
import torch
import numpy as np
import torch.nn as nn
import sys
import os
project_root = os.path.abspath("..")  # Adjust if needed
import pytorch_lightning as pl
# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.pointNetVae import PointNetVAE
%load_ext autoreload
%autoreload 2

In [116]:
def test_chamfer_dist(x_1, x_2):
    min_a_b_i = 10000000
    total_loss = 0
    array = []
    x_1 = np.array(x_1).squeeze()
    x_2 = np.array(x_2).squeeze()
    for a in x_1:
        
        array_2 = []
        for b_i in x_2:
            dist = np.sqrt(np.sum(np.abs(a - b_i)**2))
            array_2.append(dist)
            if dist < min_a_b_i:
                min_a_b_i = dist
        array.append(array_2)
        total_loss += min_a_b_i
        min_a_b_i = 100000
    return total_loss, array

In [4]:
input1 = torch.randn(128, 500, 3)
input2 = torch.randn(128, 500, 3)
output = torch.cdist(input1, input2, p = 2)

In [5]:
output_array = []
for i in range(128):
    k = test_chamfer_dist(input1[i], input2[i])
    output_array.append(k[0])

NameError: name 'test_chamfer_dist' is not defined

In [6]:
output_tensor = torch.tensor(output_array)

In [7]:
torch.sum(torch.min(output, dim= -1)[0], dim = -1) - output_tensor

RuntimeError: The size of tensor a (128) must match the size of tensor b (0) at non-singleton dimension 0

In [18]:
optimizer = torch.optim.AdamW
optimizer_param = {'lr':0.001}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PointNetVAE(2, optimizer, optimizer_param)

In [21]:
j = model(input1)

In [22]:
j[3].shape

torch.Size([128, 500, 3])